In [2]:
import sqlite3
import pandas as pd

In [13]:
def query(query):
    conn = sqlite3.connect('rpg_db.sqlite3')
    c = conn.cursor()
    c.execute(query)
    data = c.fetchall()
    return data

# Part One

## Questions to answer
1. How many total Characters are there?
2. How many of each specific subclass?
3. How many total Items?
4. How many of the Items are weapons? How many are not?
5. How many Items does each character have? (Return first 20 rows)
6. How many Weapons does each character have? (Return first 20 rows)
7. On average, how many Items does each Character have?
8. On average, how many Weapons does each character have?

In [15]:
characters = query('SELECT * FROM charactercreator_character')

In [23]:
char_cols = ['id','name','level','exp','hp','strength','intelligence','dexterity','wisdom']

In [25]:
characters = pd.DataFrame(characters,columns=char_cols)

### Character count

In [28]:
characters.count()[0]

302

### Specific subclasses

In [52]:
mage = query('''SELECT COUNT(m.character_ptr_id)
FROM charactercreator_character as c
JOIN charactercreator_mage as m
ON c.character_id = m.character_ptr_id
WHERE m.character_ptr_id NOT IN (SELECT mage_ptr_id FROM charactercreator_necromancer)''')

In [53]:
mage

[(97,)]

In [46]:
necromancer = query('''SELECT COUNT(m.character_ptr_id)
FROM charactercreator_character as c
JOIN charactercreator_mage as m
ON c.character_id = m.character_ptr_id
JOIN charactercreator_necromancer as n
ON m.character_ptr_id = n.mage_ptr_id
''')

In [47]:
necromancer

[(11,)]

In [39]:
cleric = query('''SELECT COUNT(c.name)
FROM charactercreator_character as c
JOIN charactercreator_cleric as cl
ON c.character_id = cl.character_ptr_id
''')

In [40]:
cleric

[(75,)]

In [41]:
fighter = query('''SELECT COUNT(c.name)
FROM charactercreator_character as c
JOIN charactercreator_fighter as f
ON c.character_id = f.character_ptr_id
''')

In [42]:
fighter

[(68,)]

In [44]:
thief = query('''SELECT COUNT(c.name)
FROM charactercreator_character as c
JOIN charactercreator_thief as t
ON c.character_id = t.character_ptr_id
''')

In [45]:
thief

[(51,)]

### Total items

In [55]:
query('SELECT COUNT(name) FROM armory_item')

[(174,)]

### Weapons vs not weapons

In [58]:
#weapons
query('SELECT COUNT(a.name) FROM armory_item as a JOIN armory_weapon as w ON a.item_id=w.item_ptr_id')

[(37,)]

In [61]:
# no weapons
query('SELECT COUNT(a.name) FROM armory_item as a JOIN charactercreator_character_inventory as c ON a.item_id=c.item_id')

[(898,)]

### How many Items does each character have? (Return first 20 rows)

In [71]:
items = query('''SELECT COUNT(c.item_id)
FROM charactercreator_character_inventory as c
GROUP BY c.character_id
LIMIT 20''')

In [72]:
items

[(3,),
 (3,),
 (2,),
 (4,),
 (4,),
 (1,),
 (5,),
 (3,),
 (4,),
 (4,),
 (3,),
 (3,),
 (4,),
 (4,),
 (4,),
 (1,),
 (5,),
 (5,),
 (3,),
 (1,)]

### How many Weapons does each character have? (Return first 20 rows)

In [68]:
query('''
SELECT count(items)
FROM charactercreator_character_inventory as c
LEFT JOIN
(SELECT a.item_id as items
FROM armory_item as a 
JOIN armory_weapon as w
ON items = w.item_ptr_id) 
ON c.item_id = items

GROUP BY c.character_id
LIMIT 20''')

[(0,),
 (0,),
 (0,),
 (0,),
 (2,),
 (0,),
 (1,),
 (0,),
 (0,),
 (0,),
 (1,),
 (0,),
 (0,),
 (0,),
 (0,),
 (0,),
 (0,),
 (0,),
 (0,),
 (1,)]

### On average, how many Items does each Character have?

In [73]:
query('''SELECT avg(counts) 
FROM
(SELECT COUNT(c.item_id) as counts
FROM charactercreator_character_inventory as c
GROUP BY c.character_id)''')

[(2.9735099337748343,)]

### On average, how many Weapons does each Character have?

In [70]:
query('''SELECT AVG(counts)
FROM
(SELECT COUNT(items) as counts
FROM charactercreator_character_inventory as c
LEFT JOIN
(SELECT a.item_id as items
FROM armory_item as a 
JOIN armory_weapon as w
ON items = w.item_ptr_id) 
ON c.item_id = items

GROUP BY c.character_id)''')

[(0.6721854304635762,)]

# Part Two

In [88]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)

In [89]:
df = pd.read_csv('buddymove_holidayiq.csv')

In [90]:
df.to_sql(name='review',con=engine)

In [94]:
engine.execute('SELECT count(Sports) FROM review').fetchall()

[(249,)]

In [100]:
engine.execute('''SELECT count(Sports)
FROM review as r
WHERE (r.Nature >= 100) AND (r.Shopping > 100)
''').fetchall()

[(78,)]

In [107]:
engine.execute('''SELECT AVG(Sports), AVG(Religious), AVG(Nature), AVG(Theatre), AVG(Shopping), AVG(Picnic)
FROM review as r
''').fetchall()

[(11.987951807228916, 109.77911646586345, 124.51807228915662, 116.37751004016064, 112.63855421686748, 120.40160642570281)]

In [79]:
df.head(), df.shape

(  User Id  Sports  Religious  Nature  Theatre  Shopping  Picnic
 0  User 1       2         77      79       69        68      95
 1  User 2       2         62      76       76        69      68
 2  User 3       2         50      97       87        50      75
 3  User 4       2         68      77       95        76      61
 4  User 5       2         98      54       59        95      86, (249, 7))